In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

In [38]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/PLDAC/

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/PLDAC


In [ ]:
def forget_history(l,k=4):
    mask = np.zeros(len(l),dtype=bool)
    mask[:2] = True
    mask[-k:] = True
    return np.array(l)[mask]

In [ ]:
df_canard_dev = pd.read_json('./data/canard/dev.json').drop(columns=["QuAC_dialog_id","Question_no"])
df_canard_dev_amnesia = df_canard_dev.copy()
df_canard_dev["History"] = df_canard_dev["History"].apply(lambda x: " | ".join(x))
df_canard_dev_amnesia["History"] = df_canard_dev_amnesia["History"].apply(lambda x: " | ".join(forget_history(x)))

In [ ]:
df_canard_train = pd.read_json('./data/canard/train.json').drop(columns=["QuAC_dialog_id","Question_no"])
df_canard_train_amnesia = df_canard_train.copy()
df_canard_train["History"] = df_canard_train["History"].apply(lambda x: " | ".join(x))
df_canard_train_amnesia["History"] = df_canard_train_amnesia["History"].apply(lambda x: " | ".join(forget_history(x)))

In [ ]:
df_canard_test = pd.read_json('./data/canard/test.json').drop(columns=["QuAC_dialog_id","Question_no"])
df_canard_test_amnesia = df_canard_test.copy()
df_canard_test["History"] = df_canard_test["History"].apply(lambda x: " | ".join(x))
df_canard_test_amnesia["History"] = df_canard_test_amnesia["History"].apply(lambda x: " | ".join(forget_history(x)))

In [ ]:
#df_canard_train.to_csv("./data/canard/train.csv")
#df_canard_dev.to_csv("./data/canard/dev.csv")
#df_canard_test.to_csv("./data/canard/test.csv")
df_canard_train = pd.read_csv("./data/canard/train.csv")
df_canard_dev = pd.read_csv("./data/canard/dev.csv")
df_canard_test = pd.read_csv("./data/canard/test.csv")

In [ ]:
df_canard_test.head()

,Unnamed: 0,History,Question,Rewrite
0,0,Anna Politkovskaya | The murder remains unsolv...,Did they have any clues?,Did investigators have any clues in the unreso...
1,1,Anna Politkovskaya | The murder remains unsolv...,How did they target her email?,How did FSB target the murdered journalist Ann...
2,2,Anna Politkovskaya | The murder remains unsolv...,Did they get into trouble for that?,Did FSB get into trouble for attacking the ema...
3,3,Anna Politkovskaya | The murder remains unsolv...,Did they have any murder suspects?,Did FSB have any Anna Politkovskaya's murder s...
4,4,Anna Politkovskaya | The murder remains unsolv...,Did they go to jail?,Did any of Anna Politkovskaya's murder suspect...


In [ ]:
#df_canard_train_amnesia.to_csv("./data/canard/train_amnesia.csv")
#df_canard_dev_amnesia.to_csv("./data/canard/dev_amnesia.csv")
#df_canard_test_amnesia.to_csv("./data/canard/test_amnesia.csv")
df_canard_train_amnesia = pd.read_csv("./data/canard/train_amnesia.csv")
df_canard_dev_amnesia = pd.read_csv("./data/canard/dev_amnesia.csv")
df_canard_test_amnesia = pd.read_csv("./data/canard/test_amnesia.csv")

In [39]:
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
#!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip

--2022-02-23 20:22:30--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.75.142, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 681808098 (650M) [application/zip]
Saving to: ‘wiki-news-300d-1M.vec.zip’

wiki-news-300d-1M.v 100%[===================>] 650.22M  48.0MB/s    in 14s     

2022-02-23 20:22:44 (48.0 MB/s) - ‘wiki-news-300d-1M.vec.zip’ saved [681808098/681808098]



In [ ]:
#!unzip /content/gdrive/MyDrive/PLDAC/wiki-news-300d-1M.vec.zip

Archive:  /content/gdrive/MyDrive/PLDAC/wiki-news-300d-1M.vec.zip
  inflating: wiki-news-300d-1M.vec   


In [57]:
data = defaultdict(int)

data["fafa"]

0

In [67]:
import io
from collections import defaultdict

def default_vector():
    return np.zeros(300)

def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = defaultdict(default_vector)
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = np.array(tokens[1:]).astype(float)
    return data

In [ ]:
#!head -n 200000 "/content/gdrive/MyDrive/PLDAC/wiki-news-300d-1M.vec" > "/content/gdrive/MyDrive/PLDAC/w2v_200000.vec"

In [41]:
!head -n 2 "/content/gdrive/MyDrive/PLDAC/w2v_200000.vec"

999994 300
, 0.1073 0.0089 0.0006 0.0055 -0.0646 -0.0600 0.0450 -0.0133 -0.0357 0.0430 -0.0356 -0.0032 0.0073 -0.0001 0.0258 -0.0166 0.0075 0.0686 0.0392 0.0753 0.0115 -0.0087 0.0421 0.0265 -0.0601 0.2420 0.0199 -0.0739 -0.0031 -0.0263 -0.0062 0.0168 -0.0357 -0.0249 0.0190 -0.0184 -0.0537 0.1420 0.0600 0.0226 -0.0038 -0.0675 -0.0036 -0.0080 0.0570 0.0208 0.0223 -0.0256 -0.0153 0.0022 -0.0482 0.0131 -0.6016 -0.0088 0.0106 0.0229 0.0336 0.0071 0.0887 0.0237 -0.0290 -0.0405 -0.0125 0.0147 0.0475 0.0647 0.0474 0.0199 0.0408 0.0322 0.0036 0.0350 -0.0723 -0.0305 0.0184 -0.0026 0.0240 -0.0160 -0.0308 0.0434 0.0147 -0.0457 -0.0267 -0.1703 -0.0099 0.0417 0.0235 -0.0260 -0.1519 -0.0116 -0.0306 -0.0413 0.0330 0.0723 0.0365 -0.0001 0.0042 0.0346 0.0277 -0.0305 0.0784 -0.0404 0.0187 -0.0225 -0.0206 -0.0179 -0.2428 0.0669 0.0523 0.0527 0.0149 -0.0708 -0.0987 0.0263 -0.0611 0.0302 0.0216 0.0313 -0.0140 -0.2495 -0.0346 -0.0480 0.0250 0.2130 -0.0330 -0.1553 -0.0292 -0.0346 0.1074 0.0010 -0.0117 -0.0057

In [68]:
w2v = load_vectors("/content/gdrive/MyDrive/PLDAC/w2v_200000.vec")

In [69]:
print(w2v["ajaja"])
print(np.dot(w2v["aahah"], w2v["walk"]))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
0.0


In [96]:
stemmer = SnowballStemmer("english")
key_tags = set(["CD","FW","JJ","JJR","JJS","NN","NNS","NNP","NNPS","VB","VBD","VBG","VBN","VBP","VBZ"])

def extract(s,stem=False,lower=True):
    # (to lower), tokenize, then filter by tag and remove stopword, (then stem)
    if lower:
        s = s.lower()
    if stem:
        return [stemmer.stem(w) for (w,tag) in nltk.pos_tag(nltk.word_tokenize(s)) if tag in key_tags and w not in stopwords.words('english')]
    else:
        return [w for (w,tag) in nltk.pos_tag(nltk.word_tokenize(s)) if tag in key_tags and w not in stopwords.words('english')]

def sim_overlap(s,u):
    # number of keywords in common
    return len(set(extract(s,True,False)) & set(extract(u,True,False)))

def sim_maxsim(s,u,threshold=1.5):
    # my idea
    s = set(extract(s))
    u = set(extract(u))
    sim = np.mean([np.max([np.dot(w2v[vu],w2v[vs]) for vs in s]+[0]) for vu in u]) - threshold
    return sim
    
def key_sentence(p,u_next,u="",sim=sim_maxsim):
    # sentence with highest similarity. Note: argmax favors early sentences in case of hesitation, which is a desirable behaviour.
    
    ss = p.split(".")
    scores = np.array([sim(s,u_next) for s in ss])
    if np.max(scores) > 0:
        return ss[np.argmax(scores)]
    else:
        scores = np.array([sim(s,u) for s in ss])
        if np.max(scores) > 0:
            return ss[np.argmax(scores)]
    return ""

p = "More research is needed. Types Breast cancer can be: Ductal carcinoma: This begins in the milk duct and is the most common type. Lobular carcinoma: This starts in the lobules. Invasive breast cancer is when the cancer cells break out from inside the lobules or ducts and invade nearby tissue, increasing the chance of spreading to other parts of the body. Non-invasive breast cancer is when the cancer is still inside its place of origin and has not broken out."
u_prev = "I just had a breast biopsy for cancer. What are the most common types?"
u_raw = "Once it breaks out, how likely is it to spread?"
u_auto = "Once the cancer breaks out, how likely is it to spread?"
u_manual = "Once it breaks out, how likely is lobular carcinoma breast cancer to spread?"

#print(key_sentence(p,""))
#print(key_sentence(p,"","need research"))
print(key_sentence(p,u_prev))
#print(key_sentence(p,u_raw))
#print(key_sentence(p,u_auto))
#print(key_sentence(p,u_manual))
print(key_sentence("Pros and Cons: Asphalt vs. Concrete Driveway Choosing the driveway material that’s best for you often depends on your climate and budget. When the time comes to install a new driveway, the biggest decision is whether to use concrete or asphalt. Sure, there are other driveway material choices — gravel at the low end and brick pavers at the high end — but concrete and asphalt are the most common, and each comes with advantages and disadvantages. Concrete lasts longer, but asphalt driveways are less expensive to install. Asphalt needs more maintenance, but is generally easier to repair. In winter, concrete driveways can suffer damage if you use the wrong de-icing product.","Which is cheaper way to build a driveway: concrete or asphalt?","",sim=sim_overlap))
print(key_sentence("Pros and Cons: Asphalt vs. Concrete Driveway Choosing the driveway material that’s best for you often depends on your climate and budget. When the time comes to install a new driveway, the biggest decision is whether to use concrete or asphalt. Sure, there are other driveway material choices — gravel at the low end and brick pavers at the high end — but concrete and asphalt are the most common, and each comes with advantages and disadvantages. Concrete lasts longer, but asphalt driveways are less expensive to install. Asphalt needs more maintenance, but is generally easier to repair. In winter, concrete driveways can suffer damage if you use the wrong de-icing product.","Which is cheaper way to build a driveway: concrete or asphalt?",""))

 Types Breast cancer can be: Ductal carcinoma: This begins in the milk duct and is the most common type
 When the time comes to install a new driveway, the biggest decision is whether to use concrete or asphalt
 Concrete lasts longer, but asphalt driveways are less expensive to install


In [75]:
import json

with open("./data/treccast/treccastweb-master/2021/2021_manual_evaluation_topics_v1.0.json") as json_file:
    data_2021 = json.load(json_file)

In [87]:
class TRECSet():
    def __init__(self, data):
        self.topics = [Topic(topic) for topic in data]
        self.write_history()
    
    def to_csv(self, name):
        self.df = pd.concat([topic.to_df() for topic in self.topics])
        self.df.to_csv(name)
    
    def write_history(self):
        for topic in self.topics:
            topic.write_history()

class Topic():
    def __init__(self, topic):
        self.number = topic["number"]
        self.turns = [Turn(turn) for turn in topic["turn"]]
    
    def to_df(self):
        return pd.DataFrame([turn.to_array() for turn in self.turns],\
                columns=["Question","Rewrite","History1","History2","History3",\
                "History4","History5","History6","History7","History8",\
                "History9","History10"])

    def write_history(self, extractor=None):
        history1 = []
        history2 = []
        history3 = []
        history4 = []
        history5 = []
        history6 = []
        history7 = []
        history8 = []
        history9 = []
        history10 = []
        for i in range(len(self.turns)):
            # 1> query only
            self.turns[i].history1 = " | ".join(history1)
            history1.append(self.turns[i].raw_utterance)
            # 2> recursive query only
            self.turns[i].history2 = " | ".join(history2)
            history2.append(self.turns[i].manual_rewritten_utterance)
            # 3> naive
            self.turns[i].history3 = " | ".join(history3)
            history3.append(self.turns[i].raw_utterance)
            history3.append(self.turns[i].passage)
            # 4> rewrite, top 2 summary
            ### Here i just split with . to recognize sentence. Maybe use spaCy sentence recognizer? Vincent pls do it
            self.turns[i].history4 = " | ".join(history4)
            history4.append(self.turns[i].manual_rewritten_utterance)
            history4.append(".".join(self.turns[i].passage.split(".")[:2]))

            # key sentence overlap extraction
            overlap_extracted_sentence = key_sentence(self.turns[i].passage,\
                self.turns[i+1].raw_utterance if i+1<len(self.turns) else "",\
                self.turns[i].manual_rewritten_utterance, sim=sim_overlap)

            # 5> waterloo (rewrite, overlap key sentence extraction, type b)
            self.turns[i].history5 = " | ".join(history5)
            if i > 0:
                history5[i] = ""
            history5.append(self.turns[i].manual_rewritten_utterance)
            history5.append(overlap_extracted_sentence)
            
            # 6> rewrite, overlap key sentence extraction
            self.turns[i].history6 = " | ".join(history6)
            history6.append(self.turns[i].manual_rewritten_utterance)
            history6.append(overlap_extracted_sentence)

            # 7> rewrite, overlap key sentence extraction, last 3 paragraphs
            self.turns[i].history7 = " | ".join(history7)
            if i-2 > 0:
                history7[i-2] = ""
            history7.append(self.turns[i].manual_rewritten_utterance)
            history7.append(overlap_extracted_sentence)

            # key sentence overlap extraction
            maxsim_extracted_sentence = key_sentence(self.turns[i].passage,\
                self.turns[i+1].raw_utterance if i+1<len(self.turns) else "",\
                self.turns[i].manual_rewritten_utterance, sim=sim_maxsim)
            
            # 8> maxsim key sentence extraction, last paragraph
            self.turns[i].history8 = " | ".join(history8)
            if i > 0:
                history8[i] = ""
            history8.append(self.turns[i].manual_rewritten_utterance)
            history8.append(maxsim_extracted_sentence)
            
            # 9> maxsim key sentence extraction, all paragraph
            self.turns[i].history9 = " | ".join(history9)
            history9.append(self.turns[i].manual_rewritten_utterance)
            history9.append(maxsim_extracted_sentence)

            # 10> maxsim key sentence extraction, 3 last paragraph
            self.turns[i].history10 = " | ".join(history10)
            if i-2 > 0:
                history10[i-2] = ""
            history10.append(self.turns[i].manual_rewritten_utterance)
            history10.append(maxsim_extracted_sentence)

class Turn():
    def __init__(self, turn):
        self.number = turn["number"]
        self.raw_utterance = turn["raw_utterance"]
        self.passage = turn["passage"]
        self.manual_rewritten_utterance = turn["manual_rewritten_utterance"]
        self.manual_query_history = []
        self.raw_query_history = []
        self.passage_history = []

    def to_array(self):
        return [self.raw_utterance, self.manual_rewritten_utterance,\
                self.history1, self.history2, self.history3, self.history4,\
                self.history5, self.history6, self.history7, self.history8,\
                self.history9, self.history10]

In [97]:
treccast_2021 = TRECSet(data_2021)
treccast_2021.to_csv("./data/treccast/treccastweb-master/2021/trec2021.csv")

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [98]:
df_2021 = pd.read_csv("./data/treccast/treccastweb-master/2021/trec2021.csv")

In [102]:
df_2021.head(1)

,Unnamed: 0,Question,Rewrite,History1,History2,History3,History4,History5,History6,History7,History8,History9,History10
0,0,I just had a breast biopsy for cancer. What ar...,I just had a breast biopsy for cancer. What ar...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [104]:
for i in range(1,11):
    df_2021[["Question",f"History{i}","Rewrite"]].to_csv(f"./data/treccast/treccastweb-master/2021/trec2021_{i}.csv")